# 1、字符串输出解析器 StrOutputParser

In [3]:
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser
# 1、获取大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")

print(type(response))  # AIMessage
# print(response)

# 3、如何获取一个字符串类型的结果呢？
# 方式①：自己调用输出结果的content
# print(response.content)

# 方式②：使用 StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))
print(str_response)

<class 'langchain_core.messages.ai.AIMessage'>
<class 'str'>
大语言模型（Large Language Model，LLM）是指一种通过处理大量文本数据来学习语言的高级人工智能模型。这些模型基于深度学习技术，特别是变换器（Transformer）架构，旨在理解和生成自然语言。大语言模型具有以下几个主要特点：

1. **规模庞大**：这些模型通常由数亿到数千亿个参数组成，能够捕捉语言中的复杂模式和结构。

2. **预训练和微调**：大语言模型通常首先在大型文本语料库上进行预训练，以学习语言的基本知识；然后可以通过微调适应特定的任务，如情感分析、问答、翻译等。

3. **上下文理解**：由于使用了大量的数据，语言模型能够在给定上下文的基础上生成连贯且相关的文本。

4. **应用广泛**：大语言模型可以用于各种自然语言处理（NLP）任务，包括但不限于文本生成、摘要、对话系统、信息检索等。

5. **可迁移性**：经过预训练的模型可以很容易地适应特定领域的任务，减少了从头开始训练的时间和资源消耗。

总之，大语言模型是一种强大的工具，能够在多个场景中处理和生成自然语言，为各种应用提供支持。


# 2、JSON解析器 JsonOutputParser


方式1：用户借助自己的提示词指明返回JSON格式

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
# 1、获取大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 根据提示词模板创建提示词
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个靠谱的{role}"),
        ("human", "{question}")
    ]
)

# 2、调用大模型
response = chat_model.invoke(
    chat_prompt_template.invoke(
        {
            "role": "人工智能专家",
            "question": "人工智能用英语怎么说？问题用q表示, 答案用a表示, 返回一个JSON格式的数据"
        }
    )
)

print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

<class 'langchain_core.messages.ai.AIMessage'>
```json
{
  "q": "人工智能用英语怎么说？",
  "a": "Artificial Intelligence"
}
```
{'q': '人工智能用英语怎么说？', 'a': 'Artificial Intelligence'}


方式2：借助JsonOutputParser的get_format_instructions(), 生成格式说明, 指导模型输出JSON结构

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 1、获取大模型
chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 2、定义提示词模版
# 注意，提示词模板中需要部分格式化解析器的格式要求format_instructions
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询.\n满足的格式为{format_instructions}\n问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke({"question": joke_query})

# 3、调用大模型获取返回结果
response = chat_model.invoke(prompt)
json_result = parser.invoke(response)

print(json_result)

{'joke': '为什么计算机很冷？因为它们总是打开窗口！'}


## 知识的拓展：管道符


针对于举例1改造

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
# 1、获取大模型
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

# 根据提示词模板创建提示词
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个靠谱的{role}"),
        ("human", "{question}")
    ]
)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 2、调用大模型
## 写法1：
# response = chat_model.invoke(
#     chat_prompt_template.invoke(
#         {
#             "role": "人工智能专家",
#             "question": "人工智能用英语怎么说？问题用q表示, 答案用a表示, 返回一个JSON格式的数据"
#         }
#     )
# )
# 
# json_result = parser.invoke(response)
# print(json_result)

## 写法2：
chain = chat_prompt_template | chat_model | parser
json_result_chain = chain.invoke(
    {
        "role": "人工智能专家",
        "question": "人工智能用英语怎么说？问题用q表示, 答案用a表示, 返回一个JSON格式的数据"
    }
)
print(json_result_chain)

{'q': '人工智能用英语怎么说？', 'a': 'Artificial Intelligence'}


针对于举例2改造：

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 1、获取大模型
chat_model = ChatOpenAI(
    model="gpt-4o-mini"
)

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

# 2、定义提示词模版
# 注意，提示词模板中需要部分格式化解析器的格式要求format_instructions
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询.\n满足的格式为{format_instructions}\n问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

## 写法1：
# prompt = prompt_template.invoke({"question": joke_query})
# # 3、调用大模型获取返回结果
# response = chat_model.invoke(prompt)
# json_result = parser.invoke(response)

## 写法2：
chain = prompt_template | chat_model | parser
json_result_chain = chain.invoke(
    {"question": joke_query}
)

print(json_result_chain)

{'joke': 'Why did the scarecrow win an award? Because he was outstanding in his field!'}


# 3、XML输出解析器 XMLOutputParser

举例1：自己在提示词模板中写明使用XML格式

In [2]:
from langchain_openai import ChatOpenAI
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

# 从配置文件中给环境变量重新赋值
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 获取模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 调用模型
actor_query = "周星驰的简短电影记录"
response = chat_model.invoke(f"请生成{actor_query}, 将影片附在<movie></movie>标签中")

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
周星驰（Stephen Chow）是中国香港著名的演员、导演和编剧，以其独特的喜剧风格而广受欢迎。以下是他的几部经典电影记录：

<movie>
  <title>大话西游之月光宝盒</title>
  <year>1995</year>
  <summary>周星驰出演的经典喜剧，改编自《西游记》，讲述了至尊宝和白晶晶、紫霞仙子之间的爱情故事，融入了许多搞笑和感人的元素。</summary>
</movie>

<movie>
  <title>喜剧之王</title>
  <year>1999</year>
  <summary>周星驰自导自演，讲述了一位怀揣梦想的年轻人努力成为演员的故事，展现了对梦想的执着与追寻。</summary>
</movie>

<movie>
  <title>食神</title>
  <year>1996</year>
  <summary>周星驰在片中饰演一位曾经辉煌的食神，因被竞争对手陷害而跌入低谷，随后通过努力和智慧重返巅峰，成为真正的食神。</summary>
</movie>

<movie>
  <title>逃学威龙</title>
  <year>1991</year>
  <summary>周星驰饰演的角色是一名卧底警察为了抓捕罪犯假扮学生，影片融合了校园生活与警匪元素，笑点频出。</summary>
</movie>

<movie>
  <title>西游降魔篇</title>
  <year>2013</year>
  <summary>周星驰自导自演的作品，以全新的视角重新演绎《西游记》中的经典故事，探讨了爱情与斗争的主题。</summary>
</movie>
</movie>


举例2：

In [3]:
from langchain_core.output_parsers import XMLOutputParser

parser = XMLOutputParser()
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


使用 parser.get_format_instructions() 结构实现

In [5]:
# 1.导入相关包
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 2.初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 3.测试模型的xml解析效果
actor_query = "生成汤姆·汉克斯的简短电影记录,使用中文回复"

# 4.定义XMLOutputParser对象
parser = XMLOutputParser()

# 5.生成提示词模板
prompt_template = PromptTemplate.from_template(
    template="用户的问题：{actor_query}\n使用的格式：{format_instructions}"
)

prompt = prompt_template.partial(format_instructions=parser.get_format_instructions())

prompt = prompt.invoke({"actor_query": actor_query})

# 6.调用大模型
response = chat_model.invoke(prompt)

print(response.content)

```xml
<电影记录>
   <演员>
      <姓名>汤姆·汉克斯</姓名>
      <代表作品>
         <电影>
            <标题>阿甘正传</标题>
            <年份>1994</年份>
            <简介>讲述了一个智力障碍者走过非凡人生的故事。</简介>
         </电影>
         <电影>
            <标题>拯救大兵瑞恩</标题>
            <年份>1998</年份>
            <简介>第二次世界大战期间，为了拯救一名士兵，队伍冒险穿越敌后。</简介>
         </电影>
         <电影>
            <标题>费城故事</标题>
            <年份>1993</年份>
            <简介>一名因艾滋病被解雇的律师，与社会的斗争。</简介>
         </电影>
         <电影>
            <标题>绝密飞行</标题>
            <年份>2001</年份>
            <简介>根据真实事件改编，讲述美国空军拯救被劫持飞机的故事。</简介>
         </电影>
      </代表作品>
   </演员>
</电影记录>
```


In [6]:
xml_result = parser.invoke(response)
print(xml_result)

{'电影记录': [{'演员': [{'姓名': '汤姆·汉克斯'}, {'代表作品': [{'电影': [{'标题': '阿甘正传'}, {'年份': '1994'}, {'简介': '讲述了一个智力障碍者走过非凡人生的故事。'}]}, {'电影': [{'标题': '拯救大兵瑞恩'}, {'年份': '1998'}, {'简介': '第二次世界大战期间，为了拯救一名士兵，队伍冒险穿越敌后。'}]}, {'电影': [{'标题': '费城故事'}, {'年份': '1993'}, {'简介': '一名因艾滋病被解雇的律师，与社会的斗争。'}]}, {'电影': [{'标题': '绝密飞行'}, {'年份': '2001'}, {'简介': '根据真实事件改编，讲述美国空军拯救被劫持飞机的故事。'}]}]}]}]}


# 4、列表解析器 CommaSeparatedListOutputParser

列表解析器：利用此解析器可以将模型的文本的响应转换为一个用逗号分隔的列表 (List[str]), 其实就是CSV文件格式

举例1：

In [7]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)

print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


举例2：

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 创建解析器
output_parser = CommaSeparatedListOutputParser()

# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
    " 成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# 提示模板与输出解析器传递输出
# chat_prompt =chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})

print(res)
print(type(res))

['动作片', '喜剧片', '恐怖片', '科幻片', '爱情片']
<class 'list'>


# 5、日期解析器 DatetimeOutputParser（了解）

举例1：

In [9]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 2023-07-04T14:30:00.000000Z, 1999-12-31T23:59:59.999999Z, 2025-01-01T00:00:00.000000Z

Return ONLY this string, no other words!


举例2：

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

chat_model = ChatOpenAI(model="gpt-4o-mini")
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "{format_instructions}"),
    ("human", "{request}")
])
output_parser = DatetimeOutputParser()

# 方式1：
# model_request = chat_prompt.format_messages(
# request="中华人民共和国是什么时候成立的",
# format_instructions=output_parser.get_format_instructions()
# )

# response = chat_model.invoke(model_request)
# result = output_parser.invoke(response)
# print(result)
# print(type(result))

# 方式2：
chain = chat_prompt | chat_model | output_parser
resp = chain.invoke({"request": "中华人民共和国是什么时候成立的",
                     "format_instructions": output_parser.get_format_instructions()})
print(resp)
print(type(resp))

1949-10-01 00:00:00
<class 'datetime.datetime'>
